# Exploratory Data Analysis on Top Coder data

This notebook is for exploratory analysis of data from TopCoder.com.

In [ ]:
import os
import re
import json
import pandas as pd
import numpy as np

import datetime
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.transforms as mtrans
import seaborn as sns

pd.set_option('display.max_rows', 150)

In [ ]:
with open(os.path.join(os.curdir, 'data', 'tech_by_start_date.json')) as f:
    tech_by_start_dt = json.load(f)

with open(os.path.join(os.curdir, 'data', 'number_of_track_by_date.json')) as f:
    num_of_track_by_dt = json.load(f)

with open(os.path.join(os.curdir, 'data', 'prize_of_track_by_date.json')) as f:
    prize_of_track_by_dt = json.load(f)

with open(os.path.join(os.curdir, 'data', 'number_of_dev_subtrack_by_dt.json')) as f:
    num_of_dev_subtrack_by_dt = json.load(f)

with open(os.path.join(os.curdir, 'data', 'prize_of_dev_subtrack_by_dt.json')) as f:
    prize_of_dev_subtrack_by_dt = json.load(f)

with open(os.path.join(os.curdir, 'data', 'number_of_challenges_by_project.json')) as f:
    num_of_challenges_by_project = json.load(f)

df_tech_by_dt = pd.DataFrame.from_dict(tech_by_start_dt, orient='index').fillna(0).astype(int).drop(columns='other')
df_cha_by_proj = pd.DataFrame(num_of_challenges_by_project).set_index('project_id')
df_prz_of_track_by_dt = pd.DataFrame(prize_of_track_by_dt).set_index('date')
df_num_of_track_by_dt = pd.DataFrame(num_of_track_by_dt).set_index('date')
df_prz_of_dev_subtrack_by_dt = pd.DataFrame(prize_of_dev_subtrack_by_dt).set_index('date')
df_num_of_dev_subtrack_by_dt = pd.DataFrame(num_of_dev_subtrack_by_dt).set_index('date').astype(int)

df_tech_by_dt.index = pd.to_datetime(df_tech_by_dt.index)
df_prz_of_track_by_dt.index = pd.to_datetime(df_prz_of_track_by_dt.index)
df_num_of_track_by_dt.index = pd.to_datetime(df_num_of_track_by_dt.index)
df_prz_of_dev_subtrack_by_dt.index = pd.to_datetime(df_prz_of_dev_subtrack_by_dt.index)
df_num_of_dev_subtrack_by_dt.index = pd.to_datetime(df_num_of_dev_subtrack_by_dt.index)

df_tech_by_dt.sort_index(inplace=True)
df_prz_of_dev_subtrack_by_dt.sort_index(inplace=True)
df_num_of_dev_subtrack_by_dt.sort_index(inplace=True)
df_prz_of_track_by_dt.sort_index(inplace=True)
df_num_of_track_by_dt.sort_index(inplace=True)

In [ ]:
# Util functions

def config_xtick_and_label(ax):
    """ Format the labels of xaxis where the x is a time axis."""
    year_locator = mdates.YearLocator()
    month_locator = mdates.MonthLocator()
    year_fmt = mdates.DateFormatter('%Y')
#     month_fmt = mdates.DateFormatter('%b')
    
    ax.xaxis_date()
    ax.xaxis.set_major_locator(year_locator)
    ax.xaxis.set_major_formatter(year_fmt)
    ax.xaxis.set_minor_locator(month_locator)
#     ax.xaxis.set_minor_formatter(month_fmt)
    ax.set_xlim(datetime.datetime.strptime('2014', '%Y'), datetime.datetime.strptime('2021', '%Y'))

In [ ]:
# Get all the project with more than 10 challenges under it

df_filtered_proj = \
    df_cha_by_proj.loc[df_cha_by_proj.number_of_challenges >= 10]\
    .sort_values(by='number_of_challenges', ascending=False)

fig = plt.figure(figsize=(8.5, 6), dpi=200)

with sns.axes_style('dark', {'xtick.bottom': True}):
    ax0 = fig.add_subplot(2, 1, 1)
    
    sns.distplot(df_filtered_proj.number_of_challenges, bins=20, kde=False, rug=True, ax=ax0)
    
    ax0.set_title('Number of challenges under one project\nDistribution - histogram')
    ax0.set_ylabel('Frequency')
    ax0.set_xlabel('Number of challenges')
    
    ax0.set_xticks(list(range(10, 220, 10)))
    ax0.set_xticklabels(labels=list(range(10, 220, 10)), rotation=315)
    ax0.set_ylim(bottom=0, top=200)
    
    ax0.grid(True, axis='y')
    
    for hist in ax0.patches:
        count = int(hist.get_height())
        x = hist.get_x() + hist.get_width() / 2
        y = hist.get_height()
        if count != 0:
            ax0.annotate(
                f'{count}', 
                xy=(x, y), 
                xytext=(0, 3), 
                horizontalalignment='center', 
                verticalalignment='bottom',
                textcoords='offset points'
            )

with sns.axes_style('darkgrid'):
    ax1 = fig.add_subplot(2, 1, 2)
    
    colors = {'primary': '#E93C4F', 'secondary': '#FFC24A'}
    
    meanlineprops = {'linestyle': '--', 'linewidth': 0.5, 'color': colors['secondary']}
    flierprops = {'marker': 'o', 'markerfacecolor': colors['primary'], 'markeredgewidth': 0.5, 'markersize': 2.5}
    sns.boxplot(
        x=df_cha_by_proj.loc[df_cha_by_proj.number_of_challenges > 10], 
        showmeans=True,
        meanline=True,
        
        color=colors['primary'],
        meanprops=meanlineprops,
        flierprops=flierprops,
        
        linewidth=0.8,
        width=0.618,
        
        ax=ax1
    )
    
    ax1.set_title('Number of challenges under one project\nDistribution - boxplot')
    ax1.set_xlabel('Number of challenges')

    ax1.set_xticks(list(range(20, 220, 20)))
    ax1.set_xticklabels(labels=list(range(20, 220, 20)))
    
fig.tight_layout()

## Number of challenges under a project

The two plots above visualiza the data of number of challenges under a project. The challenges data scraped from TopCoder have two data field, `challenge_id` and `project_id`. The `challenge_id` is unique identifier of a challenge. The `project_id` is unique identifier of a project. The relationship of a project and a challenge is that, _one project can have multiple challenges under it._

The visulization get rid of the datapoints less than 10. Meaning that project with less than 10 challenges under it is ignored in this visualization.

### Observation

- Shown in histogram, the projects with no more than 60 challenges have a frequency of 243, the project with more than 60 challenges have a frequency of 26.

- Shown in boxplot, the maximum data point (1.5 times of distance of IQR) is less than 60. The density of outliers in the range of 60 - aprox 80 is higher.

### Analysis.

The majority of data points falls into range of less than 60. While the density of projects with 60 to 100 challenges is apprently large than the density of projects with more than 100 challenges.

If we segregate the dataset into three tiers based on the level of adaptiveness of Crowd Source dev:

1. **Early adpater**: Projects with less than 60 challenges.
2. **Entrance level**: Projects with number of challenges between 60 to 100.
3. **Mature user**: Project with number of challenges over 100.

Based on this aggregation, it could be worth of effort to look into the requirements/complexity analysis of challenges respectively on different tiers.

**It can also be discovering to plot the scatter of projects based on start date of project (eariliest challenges)**.

If we can find pattern of challenges requriements complexity from the 3-tier separation, it will be great.

In [ ]:
# Get the summary of technology terms frequency and choose the columns for plotting
tech_sum = df_tech_by_dt.sum(axis='index').sort_values(ascending=False)

colnames = [
    list(tech_sum[tech_sum >= 800].index),
    list(tech_sum[(400 <= tech_sum) & (tech_sum < 800)].index),
    list(tech_sum[(250 <= tech_sum) & (tech_sum < 400)].index),
    list(tech_sum[(150 <= tech_sum) & (tech_sum < 250)].index),
    list(tech_sum[(100 <= tech_sum) & (tech_sum < 150)].index)
]

title_dct = {
    0: 'Technologies tagged more than 800 times',
    1: 'Technologies tagged between 400 and 800 times',
    2: 'Technologies tagged between 250 and 400 times',
    3: 'Technologies tagged between 150 and 250 times',
    4: 'Technologies tagged between 100 and 150 times\nCumulative Summary'
}

fig_type_dct = {
    0: 'Cumulative Summary',
    1: 'Month by month'
}
    
df_tech_sum_by_month = df_tech_by_dt.groupby([df_tech_by_dt.index.year, df_tech_by_dt.index.month]).sum()
df_tech_sum_by_month.index = pd.to_datetime([f'{year}-{month}' for year, month in df_tech_sum_by_month.index.to_flat_index()])
df_tech_sum_by_month.drop(df_tech_sum_by_month.loc[df_tech_sum_by_month.index > '2020-03-31'].index, inplace=True)

fig, axes = plt.subplots(10, 1, figsize=(8.5, 40), dpi=200)

with sns.axes_style('ticks'):
    for idx, ax in enumerate(axes):
        
        ax_title = '{}{}'.format(f'{title_dct[idx // 2]}\n\n' if idx % 2 == 0 else '', fig_type_dct[idx % 2])
        data = df_tech_sum_by_month[colnames[idx // 2]].cumsum() if idx % 2 == 0 else df_tech_sum_by_month[colnames[idx // 2]]
        
        sns.despine(ax=ax)
        config_xtick_and_label(ax)
        ax.set_title(ax_title)
        ax.set_xlabel('Time')
        ax.set_ylabel('Frequency')

        sns.lineplot(data=data, dashes=False, ax=ax, alpha=0.6, legend='full', linewidth=0.8)
        handles, labels = ax.get_legend_handles_labels()
        ax.legend(handles, labels, ncol=2, prop={'size': 8})
    
fig.tight_layout()

# Technology key words from challenge info

On TopCoder.com, every challenge object has a data field, `technology`. The `technology` data field is an array of strings, each element of the array is a related technology that is involved in the challenge.

The visualization above plot the cumulative summary and month by month of technology key words shown in challenges by time.

As the apperance frequency of different key words vary drastically, we manual separate the data set based on the summary of key word apperance frequecy. We consider key words appear less than 100 times in total to be too small a data point to be visualized and divide the dataset into 5 tiers:

- Technologies tagged more than 800 times
- Technologies tagged between 400 and 800 times
- Technologies tagged between 250 and 400 times
- Technologies tagged between 150 and 250 times
- Technologies tagged between 100 and 150 times

### Observation

- Except for JAVA, the technologies in the top tier (meaning that these technologies are most mentioned thus could be most demanded) are web dev technology - JavaScript, HTML, CSS, JS-based framework. 
- Technology key words appearing more than 400 times are mostly frontend technology, a lot of which are drived from JavaScript.
- Technology key words appearing less than 400 times have a lot of backend / database technology tagged.\
- Notice that `data science` is displayed as a tech key work here in range of 150 - 250 of frequency

### Analysis

- To better explore this data set, grouping the technology based on frontend/backend or similar technology catagory.
- The `data science` key word is supposed to be a track of challenges. Instead it appears in the technology stack field. This indicates that there may be a large part of noise in the data set. To address this, an analysis on the grouping pattern of technology key word will be valuable, e.g. `data science` and `python` may be tagged in the same challenges.



In [ ]:
df_track_sum_by_month = df_num_of_track_by_dt.groupby([df_num_of_track_by_dt.index.year, df_num_of_track_by_dt.index.month]).sum()
df_track_sum_by_month.index = pd.to_datetime([f'{year}-{month}' for year, month in df_track_sum_by_month.index.to_flat_index()])

df_track_prz_by_month = df_prz_of_track_by_dt.groupby([df_prz_of_track_by_dt.index.year, df_prz_of_track_by_dt.index.month]).sum()
df_track_prz_by_month.index = pd.to_datetime([f'{year}-{month}' for year, month in df_track_prz_by_month.index.to_flat_index()])

df_prz_per_cha_by_track = df_track_prz_by_month / df_track_sum_by_month
df_prz_per_cha_by_track.fillna(0)

fig = plt.figure(figsize=(8.5, 12), dpi=200)

with sns.axes_style('darkgrid', {'xtick.bottom': True}):
    ax0 = fig.add_subplot(3, 1, 1)
    sns.despine(ax=ax0)
    config_xtick_and_label(ax0)
    sns.lineplot(data=df_track_sum_by_month, dashes=False, ax=ax0, linewidth=0.75)
    ax0.set_title('Number of challenges posted every month by track')
    ax0.set_xlabel('Time (month)')
    ax0.set_ylabel('Number of challenges')
    
    ax1 = fig.add_subplot(3, 1, 2)
    sns.despine(ax=ax1)
    config_xtick_and_label(ax1)
    sns.lineplot(data=df_track_prz_by_month, dashes=False, ax=ax1, linewidth=0.75)
    ax1.set_title('Total prizes of challenges posted every month by track')
    ax1.set_xlabel('Time (month)')
    ax1.set_ylabel('Total prizes of challenges')
    
    ax2 = fig.add_subplot(3, 1, 3)
    sns.despine(ax=ax2)
    config_xtick_and_label(ax2)
    sns.lineplot(data=df_prz_per_cha_by_track, dashes=False, ax=ax2, linewidth=0.75)
    ax2.set_title('Prize per challenge every month by track')
    ax2.set_xlabel('Time (month)')
    ax2.set_ylabel('Prize per challenge')
    
fig.tight_layout()

## Challenges by tracks in a timelapse

On TopCoder.com, each challenges has a registration start time, which is the time stamp for the start of a challenge. We plot the trend of challenges number and total prize on a time axis.

### Observation

- The prize per challenge of data science track is ridiculously high comparing to develop and design tracks' challenges.
- Starting from Jan 2018, challenges in develop track had a spike on total prize, whereas challenges in design track had the total prize fell down
- The difference of number of challenges every month in develop and design track fluctuates between 50 - 300 from 2014 to 2019.
- Not until 2019 has the prize per challenge in develop track surpassed the design track in some of the months. Despite of less number of challenges and total prize, the prize per challenge in design trask has been constantly higher than develop track.

### Analysis

- The pricing pattern and complexity in Data Science should be picked out and analyze further to explore the reason why prize per challenge is so high. This may potentially indicate that the complexity in data science is gernerally higher than the other two tracks and hopefully it can link to the requirement's complexity learning.
- Same thing in design vs develop track. Why the prize per challenge in design is almost always higher than develop track regardless of the fact that develop track challenges dominate the number of challenges posted every month.

In [ ]:
df_develop_track_num_prz = pd.concat([df_track_sum_by_month.develop, df_track_prz_by_month.develop], axis=1)
df_develop_track_num_prz.columns = ['cnt', 'prize']
df_develop_track_num_prz['prz_per_cha'] = df_develop_track_num_prz.prize / df_develop_track_num_prz.cnt

df_design_track_num_prz = pd.concat([df_track_sum_by_month.design, df_track_prz_by_month.design], axis=1)
df_design_track_num_prz.columns = ['cnt', 'prize']
df_design_track_num_prz['prz_per_cha'] = df_design_track_num_prz.prize / df_design_track_num_prz.cnt

df_data_science_track_num_prz = pd.concat([df_track_sum_by_month.data_science, df_track_prz_by_month.data_science], axis=1)
df_data_science_track_num_prz.columns = ['cnt', 'prize']
df_data_science_track_num_prz['prz_per_cha'] = df_data_science_track_num_prz.prize / df_data_science_track_num_prz.cnt

df_lst = [
    {'title': 'DEVELOP', 'df': df_develop_track_num_prz},
    {'title': 'DESIGN', 'df': df_design_track_num_prz},
    {'title': 'DATA_SCIENCE', 'df': df_data_science_track_num_prz}
]

fig = plt.figure(figsize=(8.5, 9), dpi=200)

with sns.axes_style('dark', {'xitck.bottom': True}):

    for i, df in enumerate(df_lst):
        ax0 = fig.add_subplot(3, 1, i + 1)
        ax1 = ax0.twinx()
        config_xtick_and_label(ax0)
        
        sns.lineplot(data=df['df'].cnt, label='Number of challenges', ax=ax0, color='#F78562', legend=False, linewidth=0.75)
        sns.lineplot(data=df['df'].prz_per_cha, label='Prize per challenge', ax=ax1, color='#34618F', legend=False, linewidth=0.75)
    
        ax0.set_title('Challenges of track {} by month'.format(df['title']))
        ax0.set_xlabel('Time (month)')
        ax0.set_ylabel('Number of challenge')
        ax1.set_ylabel('Prize per challenge')
        ax0.grid(True, axis='x')
        
        handle0, label0 = ax0.get_legend_handles_labels()
        handle1, label1 = ax1.get_legend_handles_labels()
        ax0.legend(handle0 + handle1, label0 + label1, loc=1, prop={'size': 8})
        
fig.tight_layout()

> The visualization above is just another way to show the number of challenges and prize per challenge every month by track. 

In [ ]:
sr_dev_subtrack_sum = df_num_of_dev_subtrack_by_dt.sum().sort_values(ascending=False)
fig = plt.figure(figsize=(8.5, 3.5), dpi=200)

with sns.axes_style('darkgrid'):
    ax = fig.add_axes([0.2, 0.2, 0.8, 0.8])
    sns.barplot(x=sr_dev_subtrack_sum, y=sr_dev_subtrack_sum.index, ax=ax)
    ax.set_title('Number of DEVELOP challenges by subtrack')
    ax.set_xlabel('Number of challenges')
    ax.set_ylabel('DEVELOP Subtrack')
    ax.set_yticklabels(labels=[' '.join([w.capitalize() for w in subtrack.split('_')]) for subtrack in sr_dev_subtrack_sum.index])
    
    for bar in ax.patches:
        count = int(bar.get_width())
        x = bar.get_width()
        y = bar.get_y() + bar.get_height() / 2

        ax.annotate(
            f'{count}', 
            xy=(x, y), 
            xytext=(3, -0.5), 
            va='center',
            ha='left', 
            verticalalignment='bottom',
            textcoords='offset points'
        )

## Number of challenges in develop track by subtracks - 0

We pick out the data of challenges in develop track and group them by subtrack. The bar plot above shows the summary of number of challenges by subtrack.

### Observation

- Firt To Finsh and Code substracks takes up to 82.7% of challenges in develop track
- Except for First to Finish and Code subtracks, the Assembly Competition, UI Prototype, Bug Hunt also have more than 100 respectively

### Analysis

- An hyposis that worth of digging is that, **is there any challenge in F2F and Code that's mistakenly labelled?** e.g. Some of the Bug Hunt(testing) challenges were labelled into F2F. We should go into the requirement details to figure pick out the challenges that have testing key word in it but tagged as other subtracks.

In [ ]:
df_subtrack_num_by_month = df_num_of_dev_subtrack_by_dt.groupby([df_num_of_dev_subtrack_by_dt.index.year, df_num_of_dev_subtrack_by_dt.index.month]).sum()
df_subtrack_num_by_month.index = pd.to_datetime([f'{year}-{month}' for year, month in df_subtrack_num_by_month.index.to_flat_index()])
df_subtrack_num_by_month.drop(df_subtrack_num_by_month.loc[df_subtrack_num_by_month.index >= '2020-4-1'].index, inplace=True)

fig = plt.figure(figsize=(8.5, 8), dpi=200)

with sns.axes_style('darkgrid', {'xtick.bottom': True}):
    ax0 = fig.add_subplot(2, 1, 1)
    config_xtick_and_label(ax0)
    
    sns.lineplot(
        data=df_subtrack_num_by_month[['first_to_finish', 'code']], 
        dashes=False, 
        ax=ax0, 
        palette=['#0F8B8D', '#EEA231'], 
        alpha=0.75, 
        legend='full', 
        linewidth=1.2
    )
    ax0.set_title('Number of CODE and F2F challenges - month by month')
    ax0.set_ylabel('Number of challenges')
    ax0.set_xlabel('Time (month)')
    
    ax1 = fig.add_subplot(2, 1, 2)
    config_xtick_and_label(ax1)
    
    sns.lineplot(
        data=df_subtrack_num_by_month[['first_to_finish', 'code', 'assembly_competition', 'ui_prototype', 'bug_hunt']].cumsum(), 
        dashes=False, 
        ax=ax1, 
        palette=['#0F8B8D', '#EEA231', '#08415C', '#B8514D', '#FF773D'], 
        alpha=0.75, 
        legend='full', 
        linewidth=1.2
    )
    ax1.set_title('Number of DEVELOP challenges by subtrack - cummulative summary')
    ax1.set_ylabel('Number of challenges')
    ax1.set_xlabel('Time (month)')

fig.tight_layout()

## Number of challenges in develop track by subtracks - 1

We pick out the F2F and Code challenges to plot the number of challenges started every month, subtracks with more than 100 challenges to plot the cumulative summary.

### Observation

- Before the middle of 2015, number of chalenges each month in F2F was way higher than code. After the middle of 2015, the number of challenges in the two subtracks kept a roughly equal volumn.


### Analysis

- A preprocessing focusing on de-noising the data will be necessary - making sure that the challenges in Code subtrack have no testing and challenges in Bug Hunt subtrack have no coding.
- A price distribution of subtrack will be valuable.